In [1]:
from itertools import chain



In [2]:
#Code for Reading and preparing Excel-File

#Name of the file
file1 = "Examle of a IP list 22-10-19_smaller.xlsx" 
file2 = "Examle of a IP list 22-10-19_smaller2.xlsx" 
file3 = "Examle of a IP list 22-10-19_smaller3.xlsx" 

#read excel file
import pandas as pd

# Load the xlsx file
data1 = pd.read_excel(file1)
data2 = pd.read_excel(file2)
data3 = pd.read_excel(file3)

#merge them 
# data1.append(data2, ignore_index = True)
# data1.append(data3, ignore_index = True)
# data = data1
data = pd.concat([data1, data2,data3])
#columnnames: delete "-"
data.columns = data.columns.str.replace("-","")

#columnnames: replace double whitespace with "_"
data.columns = data.columns.str.replace("  ","_")

#columnnames: lowercase and replace whitespace with "_"
data.columns= data.columns.str.lower()
data.columns = data.columns.str.replace(" ","_")

#columnnames: replace "/" with "_or_"
data.columns = data.columns.str.replace("/","_or_")

#columnnames: delete ","
data.columns = data.columns.str.replace(",","")

#columnnames: replace "=" with "equals"
data.columns = data.columns.str.replace("=","equals")

#specific columnvalues: apply lowercase
specific_columns = ["title","assignee_or_applicant","optimized_assignee","inventor","abstract","claims",
                    "dead_or_alive_inpadoc_family_status"]

for i in range(len(specific_columns)):
    data[f"{specific_columns[i]}"]=data[f"{specific_columns[i]}"].str.lower()

#specific columnvalues: to string
specific_columns.append("inpadoc_family_members")
for i in range(len(specific_columns)):
    data[f"{specific_columns[i]}"]=data[f"{specific_columns[i]}"].astype("string")

#delete duplicate publication_numbers
data.drop_duplicates(subset="publication_number", keep='last')

#reset index
data.reset_index(inplace= True)
data = data.drop("index", axis = 1)


#change date-time-columns to date
#data['publication_date'] = pd.to_datetime(data['publication_date'])
#data['application_date'] = pd.to_datetime(data['application_date'])

In [3]:
data["inpadoc_family_members"] = data["inpadoc_family_members"].fillna("")

In [4]:
data["inpadoc_family_members"] = data.apply(lambda x: x["inpadoc_family_members"] 
                                                        if len(x["inpadoc_family_members"]) > 4 
                                                        else "",axis=1 )

In [5]:
#check if publication number has been added to inpadoc_family_members and if not add it
data["inpadoc_family_members"] = data.apply(lambda x: str(x["publication_number"])+ " | " +str(x["inpadoc_family_members"]) 
                                            if str(x["publication_number"]) not in str(x["inpadoc_family_members"]) else str(x["inpadoc_family_members"])
                                            ,axis = 1)


In [6]:
#prepare inpadoc_family_members_column
data["inpadoc_family_members"] = data.apply(lambda x: str(x["inpadoc_family_members"]).split(" | "),axis = 1)

In [7]:
#remove entries which are too short in inpadoc_family_members
for x in range(len(data)):
    cell = data.loc[x,"inpadoc_family_members"]
    for k in cell:
        if len(k) < 5:
            cell.remove(k)
    if cell != data.loc[x,"inpadoc_family_members"]:
        data.loc[x,"inpadoc_family_members"] = cell

In [8]:
type(data.loc[1,"inpadoc_family_members"])

list

In [9]:
data.loc[[32]]

,publication_number,title,assignee_or_applicant,priority_date_earliest,application_date,publication_date,inpadoc_family_members,optimized_assignee,inventor,abstract,claims,claims_(english),application_number,dead_or_alive_inpadoc_family_status,pdf_copy
32,AU1999017788A1,stabilizing elements for mechanically stabiliz...,terre armee internationale,1998-01-06,1999-01-06,1999-09-30,[AU1999017788A1],soc civ brevets vidal henri,"anderson, peter l","a modular block wall includes dry cast, unrein...",claims 1. an earthen work structure comprisin...,CLAIMS \n | 1. An earthen work structure compr...,AU199917788A,no associated inpadoc family,NaN


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18850 entries, 0 to 18849
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   publication_number                   18850 non-null  object 
 1   title                                18708 non-null  string 
 2   assignee_or_applicant                18433 non-null  string 
 3   priority_date_earliest               18837 non-null  object 
 4   application_date                     18849 non-null  object 
 5   publication_date                     18850 non-null  object 
 6   inpadoc_family_members               18850 non-null  object 
 7   optimized_assignee                   18581 non-null  string 
 8   inventor                             18295 non-null  string 
 9   abstract                             18349 non-null  string 
 10  claims                               18306 non-null  string 
 11  claims_(english)            

In [11]:
#use rerun_slower


In [12]:
data

,publication_number,title,assignee_or_applicant,priority_date_earliest,application_date,publication_date,inpadoc_family_members,optimized_assignee,inventor,abstract,claims,claims_(english),application_number,dead_or_alive_inpadoc_family_status,pdf_copy
0,DE20221860U1,einstreumischanhänger | litter mixing trailers,"hermann kirchner gmbh & co kg,de",1970-04-17,2002-08-16,2008-10-02,"[DE20221860U1, DE20114300U1, DE2018448A1, DE20...",kirchner gmbh & co kg hermann,<NA>,einstreumischfahrzeug zum verteilen und gleich...,1. einstreumischfahrzeug zum verteilen und gle...,| 1. Litter mixed vehicle for distributing and...,DE20221860U,dead,NaN
1,US6056930A,methods and compositions for mineralizing and ...,"american dental association health foundation,...",1989-05-24,1999-06-18,2000-05-02,"[US6056930A, AU199332295A, BR199206673A, CA212...",ada found,"tung, ming s.",this invention involves new compositions and m...,i claim: 1. a method for the synthesis of amo...,I claim: \n | 1. A method for the synthesis of...,US1999335888A,dead,NaN
2,US6309337B1,method of forming an insoluble phosphate miner...,"sevenson environmental services inc.,niagara f...",1990-03-16,2000-07-06,2001-10-30,"[US6309337B1, CA2118987A1, CA2118987C, GB19940...",sevenson environmental services inc,"pal, dhiraj | yost, karl w.",the present invention discloses a method of fo...,what is claimed is: 1. a method of forming i...,What is claimed is: \n | 1. A method of formi...,US2000611033A,dead,NaN
3,US7285224B2,process and compositions for the disinfection ...,"bromine compounds limited,beer sheva,il",1991-06-03,2004-12-29,2007-10-23,"[US7285224B2, AT243657T, AT342236T, AU20142136...",icl industrial products,"barak, ayala",a process for killing microorganisms and contr...,the invention claimed is: 1. a process for ki...,The invention claimed is: \n | 1. A process fo...,US200424066A,alive,NaN
4,US6336773B1,stabilizing element for mechanically stabilize...,"societe civile des brevets henri c. vidal,lepe...",1993-03-31,1999-10-14,2002-01-08,"[US6336773B1, AT220436T, AU199465224A, AU19946...",terre armee interantionale,"anderson, peter l. | cowell, michael j. | hote...","a modular block wall includes dry cast, unrein...",what is claimed is: 1. an improved mechanica...,What is claimed is: \n | 1. An improved mecha...,US1999418063A,dead,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18845,CN217210898U,underground monitoring point protecting device...,china railway fifth survey and design institut...,China Railway Fifth Survey and Design Institut...,2022-07-05,2022-07-05,"[CN217210898U, 2022-08-16]",cn217210898u,"zhou, xiao-yu | xie, zhao-hui | 周晓宇 | 谢昭晖",the utility model relates to the technical fie...,1.一种地下监测点保护装置，其特征在于，包括保护盖（1）以及与所述保护盖（1）螺接的保护筒（...,| 1. An underground monitoring point protectio...,CN202221711811U,<NA>,NaN
18846,IN202241039057A,sustainable biocement production using microbi...,"vistas,in","VISTAS,VELAN NAGAR, PV VAITHIYALINGAM ROAD, PA...",2022-07-07,2022-07-07,"[IN202241039057A, 2022-07-15]",<NA>,rajakumari k | sanjana m | thiruchelvi r | rom...,a wide range of human activities and natural p...,1. a very common effective bacterium which ind...,| 1. A very common effective bacterium which i...,IN202241039057A,<NA>,NaN
18847,CN217174744U,a negative friction eliminating device for inf...,china nonferrous metals industry kunming prosp...,China Nonferrous Metals Industry Kunming Prosp...,2022-07-11,2022-07-11,"[CN217174744U, 2022-08-12]",cn217174744u,"shi, wei | li, wen-jun | zhao, zhen-hua | song...",the utility model claims a negative friction r...,1.一种影响桩基承载力的负摩阻消除装置，其特征在于包括护筒ⅰ（1）、护筒ⅱ（2）、护筒ⅲ（3...,| 1. A negative friction resistance eliminatin...,CN202221766986U,<NA>,NaN
18848,IN202221039840A,sustainable pavement subbase material containi...,patil mahi | reddy sandeep a. | dalal parishi ...,Patil Mahi | Reddy Sandeep A. | Dalal Parishi ...,2022-07-12,2022-07-12,"[IN202221039840A, 2022-07-29]",<NA>,"patil, mahi | reddy, sandeep a. | dalal, p

In [13]:
lol = ["hs"]
type(lol) == list

True

In [14]:
print(['US1999335888A']==['US1999335888A'])

True


In [15]:
data.loc[data["publication_number"]==str('US1999335888A'),"inpadoc_family_members"]

Series([], Name: inpadoc_family_members, dtype: object)

In [16]:
data.loc[data["publication_number"]==str("US7753618B2")]

,publication_number,title,assignee_or_applicant,priority_date_earliest,application_date,publication_date,inpadoc_family_members,optimized_assignee,inventor,abstract,claims,claims_(english),application_number,dead_or_alive_inpadoc_family_status,pdf_copy


In [17]:
list(chain.from_iterable(data.loc[data["publication_number"]==str("DE20221860U1"),"inpadoc_family_members"]))

['DE20221860U1',
 'DE20114300U1',
 'DE2018448A1',
 'DE20220898U1',
 'DE20221127U1',
 'EP1306487A1']

In [18]:
x = ["US7753618B2"]
for k in x:
    if len(x) > 4:
            print("LOL")
    else:
        print("skip")
        continue
    print(len(["US7753618B2"]))


skip


In [19]:


def fractal(x,df):
    global inpadoc_undo_nested
    global inpadoc_family_members_refractal_list
    global add_members
    add_members = ""
    inpadoc_family_members_refractal_list = []
    inpadoc_undo_nested = []  
    checklist = str(df["publication_number"])
    #add inpadox to list
    #if len(x)> 1:
     #   list(chain.from_iterable(x))
    #else:
     #   list(x)
    for k in x:
        ##print(len(k))
        if len(k) > 4:
            inpadoc_family_members_refractal_list.append(k)
            #print(inpadoc_family_members_refractal_list)
        else:
            #print("skiperino")
            continue
        if k in checklist:
            #print("1:"+k)
            family_members = list(chain.from_iterable(list(df.loc[df["publication_number"]==str(k),"inpadoc_family_members"])))
            #print("HALLO"+family_members)
            for members in family_members:
                if len(members) > 4:
                    
                    inpadoc_family_members_refractal_list.append(members)
                    
                    list(chain.from_iterable(inpadoc_family_members_refractal_list))
                else:
                    continue
            #print("2:"+str(inpadoc_family_members_refractal_list))

        else:
            continue
            #print("pass")
    #print(inpadoc_family_members_refractal_list)
    return inpadoc_family_members_refractal_list
    inpadoc_undo_nested = []


In [20]:
list(chain.from_iterable(["US7753618B2","LOL"]))

['U', 'S', '7', '7', '5', '3', '6', '1', '8', 'B', '2', 'L', 'O', 'L']

In [21]:
x = ["US7753618B2","LOL"]
fractal(x,data)

['US7753618B2']

In [23]:
#test
def rerun_slower(x,df):
    global inpadoc_undo_nested
    global inpadoc_family_members_refractal_list
    inpadoc_family_members_refractal_list = []
    inpadoc_undo_nested = []  
    global run0
    run0 = fractal(x,df)
    run1 = fractal(run0,df)
    test1 = run0
    test2 = run1
    k= 0
    #print(run0)
    #print(run1)
    while set(test1) != set(test2):
        #print("hallo"+str(k))
        globals()[f"run1"] = fractal(run0,df)
        test1 = run1
        globals()[f"run0"] = fractal(run1,df)
        test2 = run0
        k=+1
        #print(k)
        #print("Startwert:"+str(globals()[f"run{k}"]))
        #print("Ergebnis:"+str(globals()[f"run{k}"]))
    return list(set(test2))

In [24]:
list(chain.from_iterable(["US7753618B2","o"]))

['U', 'S', '7', '7', '5', '3', '6', '1', '8', 'B', '2', 'o']

In [25]:
fractal(["DE20221860U1"],data)

['DE20221860U1',
 'DE20221860U1',
 'DE20114300U1',
 'DE2018448A1',
 'DE20220898U1',
 'DE20221127U1',
 'EP1306487A1']

In [26]:
test = data.head(100)
test["test"] = test.apply(lambda x: rerun_slower(x["inpadoc_family_members"],test),axis = 1)

/var/folders/tc/nxb5r5_x357f2ns4dy4fd_b40000gn/T/ipykernel_1808/1369042998.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["test"] = test.apply(lambda x: rerun_slower(x["inpadoc_family_members"],test),axis = 1)


In [27]:
test["len_inpadoc"] = test.apply(lambda x: len(x["inpadoc_family_members"]),axis = 1)
test["len_test"] = test.apply(lambda x: len(x["test"]),axis = 1)

(test["len_inpadoc"] ==test["len_test"]).unique()

/var/folders/tc/nxb5r5_x357f2ns4dy4fd_b40000gn/T/ipykernel_1808/1919662180.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["len_inpadoc"] = test.apply(lambda x: len(x["inpadoc_family_members"]),axis = 1)
/var/folders/tc/nxb5r5_x357f2ns4dy4fd_b40000gn/T/ipykernel_1808/1919662180.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["len_test"] = test.apply(lambda x: len(x["test"]),axis = 1)


array([ True])

In [28]:
test

,publication_number,title,assignee_or_applicant,priority_date_earliest,application_date,publication_date,inpadoc_family_members,optimized_assignee,inventor,abstract,claims,claims_(english),application_number,dead_or_alive_inpadoc_family_status,pdf_copy,test,len_inpadoc,len_test
0,DE20221860U1,einstreumischanhänger | litter mixing trailers,"hermann kirchner gmbh & co kg,de",1970-04-17,2002-08-16,2008-10-02,"[DE20221860U1, DE20114300U1, DE2018448A1, DE20...",kirchner gmbh & co kg hermann,<NA>,einstreumischfahrzeug zum verteilen und gleich...,1. einstreumischfahrzeug zum verteilen und gle...,| 1. Litter mixed vehicle for distributing and...,DE20221860U,dead,NaN,"[DE20221860U1, DE20221127U1, DE2018448A1, EP13...",6,6
1,US6056930A,methods and compositions for mineralizing and ...,"american dental association health foundation,...",1989-05-24,1999-06-18,2000-05-02,"[US6056930A, AU199332295A, BR199206673A, CA212...",ada found,"tung, ming s.",this invention involves new compositions and m...,i claim: 1. a method for the synthesis of amo...,I claim: \n | 1. A method for the synthesis of...,US1999335888A,dead,NaN,"[AU199332295A, US5037639A, US5460803A, US60003...",19,19
2,US6309337B1,method of forming an insoluble phosphate miner...,"sevenson environmental services inc.,niagara f...",1990-03-16,2000-07-06,2001-10-30,"[US6309337B1, CA2118987A1, CA2118987C, GB19940...",sevenson environmental services inc,"pal, dhiraj | yost, karl w.",the present invention discloses a method of fo...,what is claimed is: 1. a method of forming i...,What is claimed is: \n | 1. A method of formi...,US2000611033A,dead,NaN,"[US5193936A, US5527982A, US5569155A, US5916123...",18,18
3,US7285224B2,process and compositions for the disinfection ...,"bromine compounds limited,beer sheva,il",1991-06-03,2004-12-29,2007-10-23,"[US7285224B2, AT243657T, AT342236T, AU20142136...",icl industrial products,"barak, ayala",a process for killing microorganisms and contr...,the invention claimed is: 1. a process for ki...,The invention claimed is: \n | 1. A process fo...,US200424066A,alive,NaN,"[EP1293482A1, EP1734008A3, EP517102A1, JP03740...",82,82
4,US6336773B1,stabilizing element for mechanically stabilize...,"societe civile des brevets henri c. vidal,lepe...",1993-03-31,1999-10-14,2002-01-08,"[US6336773B1, AT220436T, AU199465224A, AU19946...",terre armee interantionale,"anderson, peter l. | cowell, michael j. | hote...","a modular block wall includes dry cast, unrein...",what is claimed is: 1. an improved mechanica...,What is claimed is: \n | 1. An improved mecha...,US1999418063A,dead,NaN,"[US5947643A, WO1994023136A3, AU701433B2, ES209...",60,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,AU1999054167A1,"nucleic acid molecule coding for beta-amylase,...",bayer cropscience gmbh,1998-07-31,1999-07-30,2000-05-04,[AU1999054167A1],bayer crop science,"frohberg, claus",the present invention relates to nucleic acid ...,we claim: 1. a nucleic acid molecule encoding...,We claim: \n | 1. A nucleic acid molecule enco...,AU199954167A,no associated inpadoc family,NaN,[AU1999054167A1],1,1
96,US6503632B1,"polydialkylsiloxane/polyamide copolymer, proce...","nof corporation,tokyo,jp",1998-08-14,2001-02-14,2003-01-07,"[US6503632B1, EP1116740A1, EP1116740A4, JP0454...",nof corp.,"hayashi, akio | shiino, daijiro | akashi, mits...",a novel polydialkylsiloxane/polyamide copolyme...,what is claimed is: 1. a polydialkylsiloxane...,What is claimed is: \n | 1. A polydialkylsilo...,US2001762962A,dead,NaN,"[EP1116740A4, WO2000009587A1, EP1116740A1, US6...",6,6
97,JP2000169848A,production of soil solidifier and solidificati...,akae yuko | kote kazuo,1998-08-24,1999-08-10,2000-06-20,"[JP2000169848A, US6638488B1]",kote kazuo | akae yuko,akae yuko | kote kazuo,problem to be solved: to accomplish soil recyc...,1. method to solidify soil using the manufactu...,| 1. Method to solidify soil using the manufac...,JP1999259277A,dead,NaN,"[JP2000169848A, US6638488B1]",2,2
98,WO2000

In [29]:
#use rerun_slower on dataframe
data["test"] = data.apply(lambda x: rerun_slower(x["inpadoc_family_members"],data),axis = 1)

In [30]:
data["len_inpadoc"] = data.apply(lambda x: len(x["inpadoc_family_members"]),axis = 1)
data["len_test"] = data.apply(lambda x: len(x["test"]),axis = 1)

In [31]:
(data["len_inpadoc"] == data["len_test"]).unique()

array([ True])

In [32]:
filter = data["len_inpadoc"] != data["len_test"]
test= data

test = test[filter]

In [33]:
#see differences in family_members
filter = data["len_inpadoc"] != data["len_test"]
test= data

test = test[filter]
test

,publication_number,title,assignee_or_applicant,priority_date_earliest,application_date,publication_date,inpadoc_family_members,optimized_assignee,inventor,abstract,claims,claims_(english),application_number,dead_or_alive_inpadoc_family_status,pdf_copy,test,len_inpadoc,len_test
